# JijModeling 1.10.0 Release Notes

JijModeling 1.10.0 provides critical bug fixes and feature enhancements to improve the stability and functionality of the library.

## Feature Enhancements

### 1-hot Constraint Detection in `Interpreter`

The `Interpreter` now identifies 1-hot constraints during constraint evaluation in the following form:

$$
\sum_{i=1}^{n} x_i = 1, \quad x_i \in \{0, 1\}
$$

This enhancement provides hints to improve solver performance for compatible solvers. Detected constraints can be accessed via the `constraint_hints` property in `ommx.v1.Instance`.

In [ ]:
import jijmodeling as jm

n = jm.Placeholder("n")
x = jm.BinaryVar("x", shape=(n,))
i = jm.Element("i", belong_to=(0,n))

problem = jm.Problem("problem")
# Add 1-hot constraint
problem += jm.Constraint("onehot", jm.sum(i, x[i]) == 1)

interpreter = jm.Interpreter({"n": 3})
instance = interpreter.eval_problem(problem)

# Display detected constraint information
print(instance.raw.constraint_hints)

Future updates will enable the detection of various constraints, including SOS1/SOS2 and n-hot constraints.

### Further Improvements in Type Hints

Type hinting across submodules has been significantly improved to resolve Pyright errors that previously occurred. This improvement leverages advancements in `pyo3-stub-gen` ([see](https://github.com/Jij-Inc/pyo3-stub-gen/releases/tag/0.6.1)) to ensure seamless type checking in submodules.

Prior to this release, type errors were common in submodules:

![image.png](./assets/jijmodeling-1.10.0_01.png)

These issues are now resolved, ensuring a more robust developer experience:

![image.png](./assets/jijmodeling-1.10.0_02.png)

## Bug Fixes

### Fixed `ndim` Calculation for Jagged Arrays

This release resolves a logic error in calculating the ndim (number of dimensions) for jagged arrays. The bug caused errors during Interpreter initialization and prevented evaluation of instances loaded through the MIPLIB loader.

In [ ]:
import jijmodeling as jm

# Load MIPLIB instance "air05"
dataset = jm.dataset.Miplib()
problem, instance_data = dataset.load("air05")

try:
    # In jijmodeling <= 1.9.0, either of these two lines would raise an InterpreterError
    interpreter = jm.Interpreter(instance_data)
    interpreter.eval_problem(problem)
except jm.InterpreterError as e:
    print(f"InterpreterError: {e}")

### Corrected Arithmetic Assignment Operator Behavior (`+=`) for Objective Functions

A long-standing issue with the `+=` operator for objective functions has been fixed. Previously, adding to an existing objective would overwrite the original instead of appending to it.

In [ ]:
import jijmodeling as jm

x = jm.BinaryVar("x")
y = jm.BinaryVar("y")

problem = jm.Problem("problem")
# Add x to objective function
problem += x
# Add y to objective function
problem += y

# In jijmodeling <= 1.9.0, the objective function would be y instead of x + y
assert jm.is_same(problem.objective, x + y)

## Changelog

### Bug Fixes

- Fixed incorrect `ndim` calculation for jagged arrays, ensuring seamless `Interpreter` initialization and MIPLIB loader evaluation.

- Resolved behavior of the `+=` operator to properly append terms to existing objective functions.

### Feature Enhancements

- 1-Hot Constraint Detection:
  Supported constraints must adhere to the following form:
    $
      \displaystyle \sum_{i=1}^{n} x_i = 1, \quad x_i \in \{0, 1\}
    $
    
  - Left-hand side: exactly one summation symbol.
  - Summation operand: a single binary variable with an index.
  - Constraint type: equality.
  - Right-hand side: a NumberLit with value of 1.
- Resolved Pyright errors in submodules by improving type hint support through `pyo3-stub-gen` enhancements.